In [2]:
import pandas as pd
import numpy as np

## 1. Korean-drama set
### 1) 데이터 로드 및 탐색

In [3]:
df1 = pd.read_csv("korean_drama.csv")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752 entries, 0 to 1751
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   kdrama_id     1752 non-null   object 
 1   drama_name    1752 non-null   object 
 2   year          1752 non-null   int64  
 3   director      1036 non-null   object 
 4   screenwriter  959 non-null    object 
 5   country       1752 non-null   object 
 6   type          1752 non-null   object 
 7   tot_eps       1752 non-null   int64  
 8   duration      1728 non-null   float64
 9   start_dt      1752 non-null   object 
 10  end_dt        1752 non-null   object 
 11  aired_on      1520 non-null   object 
 12  org_net       1344 non-null   object 
 13  content_rt    1752 non-null   object 
 14  synopsis      1584 non-null   object 
 15  rank          1752 non-null   int64  
 16  pop           1752 non-null   int64  
dtypes: float64(1), int64(4), object(12)
memory usage: 232.8+ KB


In [4]:
df1.describe()

,year,tot_eps,duration,rank,pop
count,1752.000000,1752.000000,1728.000000,1752.000000,1752.000000
mean,2019.006849,18.996005,2149.791667,22247.869292,22153.746575
std,2.317455,25.618394,1532.133619,27688.839480,37776.993814
min,2015.000000,1.000000,60.000000,9.000000,1.000000
25%,2017.000000,8.000000,720.000000,2441.500000,915.750000
50%,2019.000000,12.000000,1800.000000,6265.500000,3698.500000
75%,2021.000000,16.000000,3600.000000,49204.500000,12086.250000
max,2023.000000,150.000000,9180.000000,99999.000000,99999.000000


In [6]:
df1.isna().sum()

kdrama_id         0
drama_name        0
year              0
director        716
screenwriter    793
country           0
type              0
tot_eps           0
duration         24
start_dt          0
end_dt            0
aired_on        232
org_net         408
content_rt        0
synopsis        168
rank              0
pop               0
dtype: int64

### 2) 데이터 전처리
#### 결측치 처리
- 'director' / 'screenwriter' / 'org_net' column의 결측치 제거
- https://mydramalist.com 사이트에서 드라마 제목 검색 후 각 항목 크롤링

In [ ]:
# 드라마명 중 한글 제거
df1['drama_name'] = df1['drama_name'].str.replace(r'[가-힣]', '', regex=True)

In [ ]:
#필요없는 열 삭제
df1_filtered = df1[['drama_name', 'year', 'director', 'screenwriter',
       'tot_eps', 'duration', 'start_dt', 'org_net', 'content_rt', 'rank', 'pop']]

In [ ]:
# 'director' / 'screenwriter' / 'org_net' column 중 결측치가 하나라도 있는 행 모두 추출
unk = df1_filtered[df1_filtered.isna().any(axis=1)]

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# 크롤링
drama_names = unk["drama_name"]  

results = []

for drama_name in drama_names:
    url = f"https://mydramalist.com/search?q={drama_name.replace(' ', '+')}" 
    response = requests.get(url)


    soup = BeautifulSoup(response.content, 'html.parser')

    # 검색 결과에서 첫 번째 드라마 클릭
    element = soup.select_one(".text-primary.title a")
    if element:
        drama_page_url = "https://mydramalist.com" + element['href']
        drama_response = requests.get(drama_page_url)
        drama_soup = BeautifulSoup(drama_response.content, 'html.parser')

        # Director와 Screenwriter 찾기
        director, screenwriter, org_net = None, None, None
        directors, screenwriters = [], []

        if drama_soup.body:
            # 'Screenwriter & Director' 
            combined_tag = drama_soup.find('b', text=lambda t: t and 'Screenwriter' in t and 'Director' in t)
            if combined_tag:
                # 'Screenwriter & Director'
                combined_names = combined_tag.find_next_siblings('a')
                for name in combined_names:
                    directors.append(name.text)  
                    screenwriters.append(name.text)  

            # 'Director' 
            director_tag = drama_soup.find('b', text='Director:')
            if director_tag:
                director_name = director_tag.find_next('a').text
                directors.append(director_name)  

            # 'Screenwriter' 
            screenwriter_tag = drama_soup.find('b', text='Screenwriter:')
            if screenwriter_tag:
                screenwriter_name = screenwriter_tag.find_next('a').text
                screenwriters.append(screenwriter_name)  

            # 'Original Network' 
            org_net_tag = drama_soup.find('b', text='Original Network:')
            if org_net_tag:
                org_net = org_net_tag.find_next('a').text  

        # 결과 저장
        result = {
            "Drama Name": drama_name, 
            "Director": ', '.join(directors) if directors else None, 
            "Screenwriter": ', '.join(screenwriters) if screenwriters else None,
            "Original Network": org_net  
        }
        results.append(result)


results_df = pd.DataFrame(results)
# results_df.to_csv('added_data1.csv', index=False)

In [ ]:
results_df.columns = ['drama_name', 'director', 'screenwriter', 'org_net']

In [ ]:
# 원 데이터에 크롤링된 데이터 취합
merged_df =df1_filtered.merge(results_df[['drama_name', 'director', 'screenwriter','org_net']], on='drama_name', how='left', suffixes=('', '_from_results'))

In [ ]:
# 각 행 별 결측치에 크롤링된 값 채워넣기
merged_df['director'] = merged_df['director'].combine_first(merged_df['director_from_results'])
merged_df['screenwriter'] = merged_df['screenwriter'].combine_first(merged_df['screenwriter_from_results'])
merged_df['org_net'] = merged_df['org_net'].combine_first(merged_df['org_net_from_results'])

In [ ]:
#필요한 칼럼만 추출
merged_df = merged_df[['drama_name', 'year', 'director', 'screenwriter', 'tot_eps', 'duration',
       'start_dt', 'org_net', 'content_rt', 'rank', 'pop']]

#### 데이터타입 및 형태 통합
- 'start_dt' 칼럼 데이터타입 변환 및 형태 통합

In [ ]:
# 먼저 ISO 형식 처리
merged_df['date'] = pd.to_datetime(merged_df['start_dt'], format='%Y-%m-%d', errors='coerce')

# 이후 나머지 형식을 처리하고 NaT 값 교체
merged_df['date'] = merged_df['date'].combine_first(
    pd.to_datetime(merged_df['start_dt'], format='%b %d, %Y', errors='coerce')
)

# 최종적으로 YYYY-MM-DD 형식으로 통일한 후 YYYY-MM 값만 'date'칼럼에 저장
merged_df['date'] = merged_df['date'].dt.strftime('%Y-%m-%d').str[:7]

In [ ]:
#날짜 형이 다른 경우
# 변환 함수 정의
def convert_to_year_month(date_str):
    if pd.isna(date_str):
        return None
    
    # 공백 및 쉼표 제거
    date_str = date_str.replace(',', '').strip()
    
    # 년-월 형식으로 변환
    if len(date_str) == 4:  # 연도만 있는 경우
        return f"{date_str}-01"  # 기본 월은 01로 설정
    
    try:
        # 날짜로 변환
        date_obj = pd.to_datetime(date_str)
        return date_obj.to_period('M').strftime('%Y-%m')
    except Exception as e:
        return None  # 변환할 수 없는 경우 None 반환

In [ ]:
#날짜형 이상한 행 추출해 변환된 값 적용
date_na_val = merged_df[merged_df['date'].isna()]['start_dt'].str.split('-').str[0]
date_na_val = date_na_val.apply(convert_to_year_month)
merged_df.loc[merged_df['date'].isna(), 'date'] = date_na_val

- 'director' / 'screenwriter' 칼럼 양식 통일

In [ ]:
merged_df['director'] = merged_df['director'].astype(str).str.replace(r"[\[\]\"']", "", regex=True)
merged_df['screenwriter'] = merged_df['screenwriter'].astype(str).str.replace(r"[\[\]\"']", "", regex=True)

merged_df.loc[:, 'director'] = merged_df['director'].replace({'None': np.nan})
merged_df.loc[:, 'screenwriter'] = merged_df['screenwriter'].replace({'None': np.nan})

#### 칼럼 범주화 및 재그룹화
- 'content_rt' 칼럼 재그룹화

In [ ]:
merged_df['content_rt'] = merged_df['content_rt'].replace({
    'Not Yet Rated': 'UNK',
    '15+ - Teens 15 or older': '15',
    '18+ Restricted (violence & profanity)': '18',
    '13+ - Teens 13 or older': '13',
    'G - All Ages': 'ALL',  
    'R - Restricted Screening (nudity & violence)': '18' 
})

- 'org_net' 칼럼 범주화

In [ ]:
merged_df['org_net'] = merged_df['org_net'].str.lower()

In [ ]:
merged_df['new_org_net'] = merged_df['org_net']
merged_df['new_org_net'] = merged_df['new_org_net'].fillna('기타')

In [ ]:
broadcast_keywords = [
    'mbc', 'sbs', 'ena', 'jtbc', 'tvn', 'channel a',
    'kbs', 'tv chosun', 'obs', 'ebs', 'ocn', 'mbn',
    'drama cube', 'toonniverse', 'dramax', 'mnet',
    'e-channel', 'qtv'
]

# new_org_net 칼럼의 특정 단어가 포함된 경우 'broadcast'로 변경
merged_df['new_org_net'] = merged_df['new_org_net'].apply(
    lambda x: 'broadcast' if isinstance(x, str) and any(keyword in x.lower() for keyword in broadcast_keywords) else x
)

In [ ]:
# ott 그룹 분류
merged_df['new_org_net'] = merged_df['new_org_net'].apply(
    lambda x: 'ott' if isinstance(x, str) and not any(word in x for word in ['broadcast', '기타', 'naver', 'daum']) else x
)

In [ ]:
# web 그룹 분류
merged_df['new_org_net'] = merged_df['new_org_net'].apply(
    lambda x: 'web' if isinstance(x, str) and ('naver' in x or 'daum' in x) else x
)

#### 기타 - 'duration' 칼럼 초 > 분 단위로 변환

In [ ]:
merged_df['duration'] = merged_df['duration']/60

# 2. Review dataset
### 1) 평점 기준 단어 빈도표 생성

In [ ]:
df2 = pd.read_csv('reviews.csv')

In [ ]:
from collections import Counter
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
def pos_tagging_filter(text):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if token.pos_ in ["ADJ", "NOUN"]] 
    return filtered_tokens

In [ ]:
df2['filtered_review_text'] = df2['review_text'].apply(lambda x: pos_tagging_filter(x) if isinstance(x, str) else x)

In [ ]:
# 변경할 단어와 대체할 단어를 딕셔너리로 정의
replace_dict = {'characters': 'character', 'episodes': 'episode', 'ep': 'episode', 'actors': 'actor', 'scenes':'scene', 'writers':'writer'}

df2['filtered_review_text'] = df2['filtered_review_text'].apply(
    lambda word_list: [replace_dict.get(word, word) for word in word_list] if isinstance(word_list, list) else []
)

In [ ]:
good_df = df2[df2['overall_score']>=5]
bad_df = df2[df2['overall_score']<5]

In [ ]:
good_df['filtered_review_text'] = good_df['filtered_review_text'].apply(lambda x: x if isinstance(x, list) else [])

exclude_words = ['drama', 'dramas', 'show', 'other', 'more' ,'way', 'first', 'lead', 'bit', 'much','things','little', 'one','thing', 'last', 'own','whole', 'better',
                  'leads', 'time', 'times', 'most', 'end', 'many', 'main', 'lot', 'second', 'moment', 'moments', 'people', 'person', 'part', 'duo', 'real', 'point',
                  'side', 'fact', 'few', 'life', 'good', 'series', 'same','season', 'episode', 'scene', 'love','character', 'story']  # 제외하고 싶은 단어들 리스트

# 필터링된 단어들만 카운트
all_words = [word for sublist in good_df['filtered_review_text'] for word in sublist if word not in exclude_words]
word_count = Counter(all_words)
good_count_df = pd.DataFrame(word_count.items(), columns=['word', 'count'])

good_count_df = good_count_df.sort_values(by='count', ascending=False).reset_index(drop=True)

In [ ]:
bad_df['filtered_review_text'] = bad_df['filtered_review_text'].apply(lambda x: x if isinstance(x, list) else [])

# 필터링된 단어들만 카운트
all_words = [word for sublist in bad_df['filtered_review_text'] for word in sublist if word not in exclude_words]
word_count = Counter(all_words)
bad_count_df = pd.DataFrame(word_count.items(), columns=['word', 'count'])

bad_count_df = bad_count_df.sort_values(by='count', ascending=False).reset_index(drop=True)

In [ ]:
good_count_df['sep']='good'
bad_count_df['sep']='bad'

word_count_df = pd.concat([good_count_df, bad_count_df], axis=0, ignore_index=True)

word_count_df.to_csv('review_word_count_fin.csv', index=False)

# 3. 신규 데이터프레임 생성
### 1) Actor

In [ ]:
df3= pd.read_csv('wiki_actors.csv')

In [ ]:
#데이터 컬럼 정리(각 데이터셋에 필요없는 컬럼들 정리)
df3 = df3.drop(columns= ['actor_id', 'character_name', 'role'])
df2 = df2.drop(columns=['user_id', 'review_text', 'ep_watched', 'n_helpful'])

# df3[drama_name] 항목에 텍스트 조정
df3['drama_name'] = df3['drama_name'].str.replace(r'[가-힣]', '', regex=True)

# 데이터프레임 병합 (drama_name과 title을 기준으로 병합)
actor_merged_df = pd.merge(df3, df2, left_on='drama_name', right_on='title', how='left')

# title 열 삭제 (drama_name과 같은 값이므로 중복 제거)
actor_merged_df = actor_merged_df.drop(columns=['title'])

# actor_name과 drama_name으로 그룹화하고 나머지 평점의 평균 계산
actor_grouped_df = actor_merged_df.groupby(['actor_name', 'drama_name']).mean().reset_index()

# 모든 평점 컬럼의 값을 소수점 첫 번째 자리로 반올림
actor_grouped_df = actor_grouped_df.round({
    'story_score': 1,
    'acting_cast_score': 1,
    'music_score': 1,
    'rewatch_value_score': 1,
    'overall_score': 1
})

#결측치 제거
actor_grouped_df = actor_grouped_df.dropna()
actor_grouped_df.info()


#csv 파일로 저장
actor_grouped_df.to_csv('actor_grouped_around_df.csv', index=False)

### 2) Director

In [ ]:
director_df = merged_df.drop(columns= ['year', 'screenwriter', 'tot_eps','duration','screenwriter','org_net','content_rt','date'])

# 행분리 (director 기준)
merged_df_explode = director_df.assign(director=merged_df.director.str.split(',')).explode('director')
# 데이터프레임 병합 (drama_name과 title을 기준)
director_merged_df = pd.merge(merged_df, df2, left_on='drama_name', right_on='title', how='left')


In [ ]:
# 중복 값 title 제거 ( = drama_name 과 동일)
director_merged_df = director_merged_df.drop(columns=['title'])
# director와 drama_name으로 그룹화하고 나머지 평점의 평균 계산
director_grouped_df = director_merged_df.groupby(['director', 'drama_name']).mean().reset_index()

# rank와 pop 칼럼은 정수형으로 변환하고, 나머지 평점 관련 칼럼은 소수점 첫째 자리까지 반올림
director_grouped_df['rank'] = director_grouped_df['rank'].round(0).astype(int)
director_grouped_df['pop'] = director_grouped_df['pop'].round(0).astype(int)

# 나머지 평점 칼럼은 소수점 첫째 자리까지 반올림
director_grouped_df[['story_score', 'acting_cast_score', 'music_score', 'rewatch_value_score', 'overall_score']] = director_grouped_df[['story_score', 'acting_cast_score', 'music_score', 'rewatch_value_score', 'overall_score']].round(1)
# 각 행에 결측치가 하나라도 있는 경우 제거
director_grouped_df = director_grouped_df.dropna(how='any')

# CSV 파일로 저장
director_grouped_df.to_csv('director_grouped.csv', index=False)

### 3) Writer

In [ ]:
df = pd.read_csv('new_org_drama.csv')
df_exploded = df.assign(screenwriter=df['screenwriter'].str.split(', ')).explode('screenwriter')
df_merged = df2.groupby('title')[['story_score','acting_cast_score','music_score','rewatch_value_score','overall_score']].mean().reset_index()
writer_merged_df = pd.merge(df_exploded, df_merged, left_on='drama_name', right_on='title', how='left')

writer_merged_df = writer_merged_df[['drama_name','screenwriter',
       'story_score', 'acting_cast_score', 'music_score',
       'rewatch_value_score', 'overall_score']]

writer_merged_df = writer_merged_df.round({
    'story_score': 1,
    'acting_cast_score': 1,
    'music_score': 1,
    'rewatch_value_score': 1,
    'overall_score': 1
})

writer_merged_df = writer_merged_df.dropna()
writer_merged_df.to_csv('writer_finish_df.csv', index=False)

# 4. 사용자 특성 데이터

In [ ]:
df = pd.read_csv(f"CI_MOBILE_OTT_WTCHNG_GENRE_ND_USR_CHARTR_INFO_202305.csv")

In [ ]:
df.drop(['RESPOND_ID', 'EXAMIN_BEGIN_DE'],axis=1,inplace=True)

In [ ]:
df.columns = ['국내영화',
              '해외영화',
              '국내드라마',
              '해외드라마',
              '예능프로그램',
              '다큐멘터리',
              '애니메이션',
              '생방송',
              '키즈',
              '성별',
              '연령대',
              '결혼여부',
              '가구소득정도',
              '직업명',
              '거주지역']

df[['국내영화','해외영화','국내드라마','해외드라마','예능프로그램','다큐멘터리','애니메이션','생방송','키즈']] = df[['국내영화','해외영화','국내드라마','해외드라마','예능프로그램','다큐멘터리','애니메이션','생방송','키즈']].replace({'Y': 1, 'N': 0})

In [ ]:
df['이용여부'] = df[['국내영화','해외영화','국내드라마','해외드라마','예능프로그램','다큐멘터리','애니메이션','생방송','키즈']].apply(lambda row: 1 if row.sum() > 0 else 0, axis=1)

In [ ]:
# 지역 그룹화
region_mapping = {
    '서울': '수도권',
    '경기': '수도권',
    '인천': '수도권',
    '충남': '충청도',
    '충북': '충청도',
    '대전': '충청도',
    '세종': '충청도',
    '경남': '경상도',
    '경북': '경상도',
    '부산': '경상도',
    '대구': '경상도',
    '울산': '경상도',
    '광주': '전라도',
    '전북': '전라도',
    '전남': '전라도',
    '제주': '제주',
    '강원': '강원도'
}

# 새로운 지역 범주로 변환
df['지역권'] = df['거주지역'].replace(region_mapping)

## 중위연령

In [ ]:
pop = pd.read_csv('인구.csv', encoding="ANSI")

In [ ]:
pop_df = pop[pop['인구구조, 부양비별(1)'] == '중위연령(세)'][['시도별(1)','2024']][1:]
pop_df.columns = ['지역', '중위연령']

In [ ]:
pop_df['지역'] = pop_df['지역'].replace({
    '서울특별시' : '서울',
    '부산광역시':'부산',
    '대구광역시':'대구',
    '인천광역시':'인천',
    '광주광역시':'광주',
    '대전광역시':'대전',
    '울산광역시':'울산',
    '세종특별자치시':'세종',
    '경기도':'경기',
    '강원도':'강원',
    '충청북도':'충북',
    '충청남도':'충남',
    '전라북도':'전북',
    '전라남도':'전남',
    '경상북도':'경북',
    '경상남도':'경남',
    '제주특별자치도':'제주'
})

In [ ]:
df_fin = pd.merge(df, pop_df, left_on='거주지역', right_on='지역', how='left')

In [ ]:
df_fin = df_fin[['국내드라마','연령대', '거주지역', '이용여부', '소득범주', '지역권', '지역', '중위연령']]

In [ ]:
df_fin.to_csv('ott_watcher_fin.csv', index=False)